In [1]:
# https://dplyr.tidyverse.org/reference/context.html
from datar.all import *

%run nb_helpers.py
nb_header(cur_group_id, cur_group_rows, cur_data, cur_data_all, cur_column, book='context')

[2021-07-16 15:28:23][datar][WARNING] Builtin name "min" has been overriden by datar.
[2021-07-16 15:28:23][datar][WARNING] Builtin name "max" has been overriden by datar.
[2021-07-16 15:28:23][datar][WARNING] Builtin name "sum" has been overriden by datar.
[2021-07-16 15:28:23][datar][WARNING] Builtin name "abs" has been overriden by datar.
[2021-07-16 15:28:23][datar][WARNING] Builtin name "round" has been overriden by datar.
[2021-07-16 15:28:23][datar][WARNING] Builtin name "all" has been overriden by datar.
[2021-07-16 15:28:23][datar][WARNING] Builtin name "any" has been overriden by datar.
[2021-07-16 15:28:23][datar][WARNING] Builtin name "re" has been overriden by datar.
[2021-07-16 15:28:23][datar][WARNING] Builtin name "filter" has been overriden by datar.
[2021-07-16 15:28:23][datar][WARNING] Builtin name "slice" has been overriden by datar.


### # cur_group_id  

##### gives a unique numeric identifier for the current group.


### # cur_group_rows  

##### Gives the row indices for the current group.

##### Args:
&emsp;&emsp;`_data`: The dataFrame.  

##### Returns:
&emsp;&emsp;The `_rows` from group data or row indexes (always 0-based).  


...

##### gives the current data for the current group
(excluding grouping variables).  


### # cur_data_all  

##### gives the current data for the current group
(including grouping variables)  


### # cur_column  

##### Used in the functions of across. So we don't have to register it.


In [2]:
df = tibble(
  g=['a'] + ['b'] * 2 + ['c'] * 3,
  x=runif(6),
  y=runif(6)
)
gf = df >> group_by(f.g)

gf >> summarise(n = n())

,g,n
,<object>,<int64>
0,a,1
1,b,2
2,c,3


In [3]:
gf

,g,x,y
,<object>,<float64>,<float64>
0,a,0.355023,0.000239
1,b,0.880160,0.685917
2,b,0.346984,0.925864
3,c,0.235711,0.701329
4,c,0.320924,0.903383
5,c,0.517224,0.223750


In [4]:
gf >> mutate(id=cur_group_id()) 

,g,x,y,id
,<object>,<float64>,<float64>,<int64>
0,a,0.355023,0.000239,0
1,b,0.880160,0.685917,1
2,b,0.346984,0.925864,1
3,c,0.235711,0.701329,2
4,c,0.320924,0.903383,2
5,c,0.517224,0.223750,2


In [5]:
gf >> summarise(row=cur_group_rows()) 

[2021-07-16 15:28:24][datar][   INFO] `summarise()` has grouped output by ['g'] (override with `_groups` argument)


,g,row
,<object>,<int64>
0,a,0
1,b,1
2,c,3
3,b,2
4,c,4
5,c,5


In [6]:
gf_group = gf >> summarise(data=[cur_group()])
gf_group 

,g,data
,<object>,<object>
0,a,<DF 1x1>
1,b,<DF 1x1>
2,c,<DF 1x1>


In [7]:
gf_group >> pull(f.data, to='list')

[         g
   <object>
 0        a,
          g
   <object>
 1        b,
          g
   <object>
 2        c]

In [8]:
gf_data = gf >> summarise(data=[cur_data()])
gf_data

,g,data
,<object>,<object>
0,a,<DF 1x2>
1,b,<DF 2x2>
2,c,<DF 3x2>


In [9]:
gf_data >> pull(f.data, to='list')

[          x         y
   <float64> <float64>
 0  0.355023  0.000239,
           x         y
   <float64> <float64>
 0  0.880160  0.685917
 1  0.346984  0.925864,
           x         y
   <float64> <float64>
 0  0.235711  0.701329
 1  0.320924  0.903383
 2  0.517224  0.223750]

In [10]:
gf_data_all = gf >> summarise(data=[cur_data_all()])
gf_data_all

,g,data
,<object>,<object>
0,a,<DF 1x3>
1,b,<DF 2x3>
2,c,<DF 3x3>


In [11]:
gf_data_all >> pull(f.data, to='list')

[         g         x         y
   <object> <float64> <float64>
 0        a  0.355023  0.000239,
          g         x         y
   <object> <float64> <float64>
 0        b  0.880160  0.685917
 1        b  0.346984  0.925864,
          g         x         y
   <object> <float64> <float64>
 0        c  0.235711  0.701329
 1        c  0.320924  0.903383
 2        c  0.517224  0.223750]

In [12]:
df >> select(f.x, f.y) >> mutate(
    across(
        everything(), 
        lambda x, cc: [cc + ' '] * x.shape[0] + (x**2).astype(str), cc=cur_column()
    )
)

,x,y
,<object>,<object>
0,x 0.12604155322312222,y 5.6999380937828065e-08
1,x 0.774681491948246,y 0.4704825471271767
2,x 0.12039757121582166,y 0.8572243453665154
3,x 0.05555990226957967,y 0.4918620958711894
4,x 0.10299227187650134,y 0.8161000953420035
5,x 0.2675202810065084,y 0.05006392781773493


In [13]:
# or you can use x.name, since x is a Series
df >> mutate(across(
    [f.x, f.y], 
    lambda x: [x.name + ' '] * x.shape[0] + (x**2).astype(str)
))

,g,x,y
,<object>,<object>,<object>
0,a,x 0.12604155322312222,y 5.6999380937828065e-08
1,b,x 0.774681491948246,y 0.4704825471271767
2,b,x 0.12039757121582166,y 0.8572243453665154
3,c,x 0.05555990226957967,y 0.4918620958711894
4,c,x 0.10299227187650134,y 0.8161000953420035
5,c,x 0.2675202810065084,y 0.05006392781773493
